# QAOA 
# (Quantum Approximate Optimazation Algorithm: 量子近似最適化アルゴリズム)

## 概要

　量子コンピュータで最適化というと、量子アニーラーのイメージが強いが、ゲート型でも特定の最適化問題は解くことができる。QAOAとは、そんなゲート型量子コンピュータを用いて、組み合わせ最適化問題(最適な組み合わせを選ぶ、最適化問題の一種)の解を求めるためのアルゴリズムである。

## 数学的説明

最適な解を1番ハミルトニアン(エネルギー演算子)の値が最も小さい状態とすると、以下のようなシュレディンガー方程式が書ける。　

$-i\frac{\partial}{\partial t}|\psi\rangle = \hat{H}(t)|\psi\rangle$ (今後の計算のために$\hbar$は省略している)

ここで、[1]より $|\psi\rangle = exp(- i\hat{H}(t))|\psi\rangle$と導け、$\hat{U}(t) = exp(- i\hat{H}(t))$とおくと、$|\psi\rangle = \hat{U}(t)|\psi\rangle$と言える。ちなみに、この$\hat{U}(t)$は時間発展演算子という。

そして、両辺の演算子の関係は以下のようになる。$-i\frac{\partial}{\partial t}\hat{U}(t) = \hat{H}(t)\hat{U}(t)$ 

よって、$\hat{U}(t) = \frac{1}{i}\int_0^t \hat{H}\hat{U}(t) dt + \hat{C} = -i\int_0^t \hat{H}\hat{U}(t) dt + \hat{C} $ 

ここで、$t = 0$の時$\hat{C} = \hat{U}(t) = 1$より、$\hat{U}(t) = 1 -i \int_0^t \hat{H}\hat{U}(t') dt'$と書き換えられる。

よく見ると、両辺に$\hat{U}(t)$と$\hat{U}(t')$という同じ関数が入っているので、式を展開すると、

$\hat{U}(t) = 1 -i \int_0^t \hat{H}(t')dt' + (-i)^{2}\int_0^t \int_0^{t'}\hat{H}(t')\hat{H}(t'')\hat{U}(t'') dt''dt'$と書き換えられる。　

よって、これを無限に積分すると、$\hat{U}(t) = 1 -i \int_0^t \hat{H}(t')dt' + (-i)^{2}\int_0^t \int_0^{t'}\hat{H}(t')\hat{H}(t'')\hat{U}(t'') dt''dt' + (-i)^{3}\int_0^t \int_0^{t'} \int_0^{t''} \hat{H}(t')\hat{H}(t'') \hat{H}(t''')\hat{U}(t''') dt''' dt''dt' + \cdots$と展開できる。

よって、$\hat{U}(t) = \exp(\int_0^t -i\hat{H}(t) dt) = \exp(-i\hat{H}(t) dt)\exp(-i\hat{H}(t-dt) dt) \cdots \exp(-i\hat{H}(dt) dt)\exp(-i\hat{H}(0) dt)$と表せる。　

ここで、２つのハミルトニアンは非可換なのでトロッター展開[2]より、
$\exp(-i\hat{H}(dt) dt) \approx \exp(-i\hat{H}(1-t)(dt) dt) \exp(-i\hat{H}t(dt) dt)$である。

更に、$\beta = (1-t)dt, \gamma = tdt$とおくと、$H_{ref}$の量子状態に$U(t) = \prod_{i=1}^p \exp(-i\beta_{i} \hat{H}_{ref})\exp(-i\gamma_{i} \hat{H}_{cost})$を作用させると最適解に近づく。

## アルゴリズムの手順

ここでは繰り返し回数が1回の時の手順を示す。繰り返し回数とは、以下のステップ2と3を行う回数を指す。

1. 重ね合わせ状態$|+\rangle^{\otimes n}$を作る。($n$は量子レジスタ数) 

2. 任意の角度$\beta, \gamma$を選び、量子ビットのペア全てに$CNOT_{i,j}(i \neq j),R_{z}(2\gamma),CNOT_{i,j}(i \neq j)$を掛ける。[3] 

3. 各量子レジスタに$R_{x}(2\beta)$を掛ける。  [3]

4. $\langle \beta, \gamma |H_0 |\beta, \gamma \rangle$ を最小にする$\beta, \gamma$ を古典コンピュータで求める。

5. 2-4を繰り返し、複数回の試行の中で$\langle \beta, \gamma |H_0 |\beta, \gamma \rangle$が最小になった時の$\beta, \gamma$で1-3を繰り返し、量子レジスタを測定する。　

## Maxcut問題の説明
![220px-Min-cut svg](https://user-images.githubusercontent.com/45162150/54339910-82be2f80-4679-11e9-9b9c-a8eab23e1d2d.png)(引用元[5])

Maxcut問題とは、重み付きのノード(頂点)$N$を２つに分割する問題で、分割されるエッジの総和を最大化する問題である。[6] 
分割する一方のグループの重みを1,もう片方の重みを-1とすると、
$$ C_{\alpha} = \sum_{i,j\in N}^{} (1-\sigma_{i}\sigma_{j})$$を最大化する問題である。　

## 実装
今からQAOAを株式会社Qunasysが開発してる量子シミュレータQulacsを用いて実装していく。今回はQAOAを用いてMaxcut問題を解いていく。

In [10]:
from qulacs import QuantumState,QuantumCircuit,ParametricQuantumCircuit,Observable
from qulacs.gate import H,CNOT,RX,RZ
from scipy.optimize import minimize
import numpy as np

In [16]:
n = 5 
state = QuantumState(n)
state.set_zero_state()

circuit = QuantumCircuit(n)

for i in range(1,n+1):
    circuit.add_H_gate(i)
    
def gamma_circuit(circuit,gamma,i,j):
    circuit.add_gate(CNOT(i,j))
    circuit.add_gate(RZ(j,2*gamma))
    circuit.add_gate(CNOT(i,j))
    
def beta_circuit(circuit, beta,i):
    circuit.add_gate(RX(i,2*beta))
    
def observable(beta, gamma):
    for i in range(1,n+1):
        gamma_circuit(circuit,gamma,i,(i+1)%4)
        beta_circuit(circuit, beta,i)
        
    circuit.update_quantum_state(state)
    observable.get_expectation_value(state)
    
minimize(observable(0,0), x0=[(0,np.pi),(0,2*np.pi)], method='Nelder-Mead')

AttributeError: 'function' object has no attribute 'get_expectation_value'